In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from tqdm import tqdm

In [2]:
ooflgb = pd.read_hdf('lgboof.h5', key='data')
oofnn1 = pd.read_hdf('nn1oof.h5', key='data')
# oofnn2 = pd.read_hdf('nn2oof.h5', key='data')
# oofnn3 = pd.read_hdf('nn2oof2.h5', key='data')

In [3]:
all_data = pd.concat([ooflgb,oofnn1],axis=1)#,oofnn2,oofnn3

In [4]:
all_data.columns = ['a','b']#,'c','d'

In [5]:
all_data.shape

(42907133, 4)

In [6]:
train = all_data.iloc[0:42907133-1000000]
test = all_data.iloc[42907133-1000000:]

In [7]:
param = {'num_leaves': 52,
         'objective':'binary',
         'learning_rate': 0.01,        
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8 ,
         "bagging_seed": 11,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "random_state": 133,
         "verbosity": -1,
         "num_threads" : -1,

         }

In [8]:
target = pd.read_hdf('train_data.h5', key='data')['label']

In [9]:
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold,RepeatedKFold
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=15)
stack_test = np.zeros((len(test),1))

In [10]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, target)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx],
                           label=target[trn_idx],
                           )
    val_data = lgb.Dataset(train.iloc[val_idx],
                           label=target[val_idx],
                           )

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets=[trn_data, val_data],
                    verbose_eval=50,
                    early_stopping_rounds=100)
    stack_test += clf.predict(test, num_iteration=clf.best_iteration).reshape(-1,1)


fold n°0
Training until validation scores don't improve for 100 rounds
[50]	training's auc: 0.806357	valid_1's auc: 0.806748
[100]	training's auc: 0.807591	valid_1's auc: 0.807936
[150]	training's auc: 0.808344	valid_1's auc: 0.808655
[200]	training's auc: 0.808833	valid_1's auc: 0.809105
[250]	training's auc: 0.809044	valid_1's auc: 0.809282
[300]	training's auc: 0.809235	valid_1's auc: 0.809436
[350]	training's auc: 0.809398	valid_1's auc: 0.809554
[400]	training's auc: 0.809514	valid_1's auc: 0.80963
[450]	training's auc: 0.809594	valid_1's auc: 0.809669
[500]	training's auc: 0.809667	valid_1's auc: 0.809697
[550]	training's auc: 0.809739	valid_1's auc: 0.809716
[600]	training's auc: 0.809805	valid_1's auc: 0.809728
[650]	training's auc: 0.809869	valid_1's auc: 0.809734
[700]	training's auc: 0.809929	valid_1's auc: 0.809737
[750]	training's auc: 0.809982	valid_1's auc: 0.80974
[800]	training's auc: 0.810032	valid_1's auc: 0.809744
[850]	training's auc: 0.810082	valid_1's auc: 0.8097

KeyboardInterrupt: 

In [11]:
stack_test = pd.DataFrame(stack_test/5).reset_index()
stack_test.columns=['id','probability']
stack_test['id']+=1
stack_test.to_csv('submission.csv',index=False)